In [20]:
import json
import uuid
from sqlalchemy import create_engine

from utils import reset_db, get_session, model_to_dict
from data.models import udahub

# Udahub Application

## Core Database

**Init DB**

In [21]:
udahub_db = "data/core/udahub.db"

In [22]:
reset_db(udahub_db)

✅ Removed existing data/core/udahub.db
2025-10-06 14:51:12,008 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-06 14:51:12,008 INFO sqlalchemy.engine.Engine COMMIT
✅ Recreated data/core/udahub.db with fresh schema


In [23]:
engine = create_engine(f"sqlite:///{udahub_db}", echo=False)
udahub.Base.metadata.create_all(bind=engine)

**Account**

In [24]:
account_id = "cultpass"
account_name = "CultPass Card"

In [25]:
with get_session(engine) as session:
    account = udahub.Account(
        account_id=account_id,
        account_name=account_name,
    )
    session.add(account)

## Integrations

**Knowledge Base**

In [26]:
# Write generated articles to JSONL (append, dedupe by title)
import os

jsonl_path = 'data/external/cultpass_articles.jsonl'

# Ensure we have the generated list available
if 'additional_articles' not in globals():
    additional_articles = [
        {"title": "Understanding Quotas and Rollovers", "content": "Each plan grants a monthly quota of 4 experiences. Unused quota does not roll over.", "tags": "quota, usage, policy"},
        {"title": "Update App and Clear Cache", "content": "Update from the app store. If issues persist, clear cache from Settings > Storage.", "tags": "app, technical, troubleshooting"},
        {"title": "Change Password and Manage 2FA", "content": "Use 'Forgot Password' to reset. Enable 2FA under Security.", "tags": "security, password, 2fa"},
        {"title": "Payment Methods and Receipts", "content": "Manage cards in Billing. Receipts are emailed and invoices in Billing History.", "tags": "billing, payments, receipts"},
        {"title": "Refund Eligibility", "content": "Refunds for duplicates or canceled events. Quota refunds not offered.", "tags": "billing, refunds, policy"},
        {"title": "Switch Subscription Tier", "content": "Upgrades prorate immediately. Downgrades at next billing cycle.", "tags": "subscription, upgrade, downgrade"},
        {"title": "QR Code Troubleshooting at Venue", "content": "Increase brightness, update app, refresh code, or use backup alphanumeric code.", "tags": "qr, access, venue"},
        {"title": "Late Arrival and No-Show", "content": "Held 15 minutes after start. No-shows count toward quota.", "tags": "policy, reservations, attendance"},
        {"title": "Group Reservations", "content": "Owner invites members from reservation screen. Quotas deducted per attendee.", "tags": "group, reservations, quotas"},
        {"title": "Contact Human Support", "content": "In-app chat > 'Talk to a person' or email support@cultpass.example with ticket ID and device.", "tags": "support, escalation, contact"},
    ]

# Read existing titles to avoid duplicates
existing_titles = set()
if os.path.exists(jsonl_path):
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                try:
                    existing_titles.add(json.loads(line)["title"]) 
                except Exception:
                    pass

rows_to_append = [a for a in additional_articles if a["title"] not in existing_titles][:10]

if rows_to_append:
    with open(jsonl_path, 'a', encoding='utf-8') as f:
        for row in rows_to_append:
            f.write(json.dumps(row, ensure_ascii=False) + "\n")
    print(f"Appended {len(rows_to_append)} articles to {jsonl_path}")
else:
    print("No new articles appended (all titles present).")

# Show final count
final = 0
with open(jsonl_path, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            final += 1
print('Total articles now:', final)


No new articles appended (all titles present).
Total articles now: 14


In [27]:
cultpass_articles = []

with open('data/external/cultpass_articles.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_articles.append(json.loads(line))

In [28]:
cultpass_articles

[{'title': 'How to Reserve a Spot for an Event',
  'content': 'If a user asks how to reserve an event:\n\n- Guide them to the CultPass app\n- Instruct them to browse the experience catalog and tap \'Reserve\'\n- If it\'s a premium or limited event, check if reservation confirmation is required via email\n- Remind them to arrive at least 15 minutes early with their QR code visible\n\n**Suggested phrasing:**\n"You can reserve an experience by opening the CultPass app, selecting your desired event, and tapping \'Reserve\'. Be sure to arrive 15 minutes early with your QR code ready."',
  'tags': 'reservation, events, booking, attendance'},
 {'title': "What's Included in a CultPass Subscription",
  'content': 'Each user is entitled to 4 cultural experiences per month, which may include:\n- Art exhibitions\n- Museum entries\n- Music concerts\n- Film screenings and more\n\nSome premium experiences may require an additional fee (visible in the app).\n\n**Suggested phrasing:**\n"Your CultPass s

In [29]:
if len(cultpass_articles) < 14:
    raise AssertionError("You should load the articles with at least 14 records")

In [30]:
with get_session(engine) as session:
    kb = []
    for article in cultpass_articles:
        knowledge = udahub.Knowledge(
            article_id=str(uuid.uuid4()),
            account_id=account_id,
            title=article["title"],
            content=article["content"],
            tags=article["tags"]
        )
        kb.append(knowledge)
    session.add_all(kb) 
    

**Ticket**

In [31]:
cultpass_users = []

with open('data/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

In [32]:
ticket_info = {
    "status": "open",
    "content": "I can't log in to my Cultpass account.",
    "owner_id": cultpass_users[0]["id"],
    "owner_name": cultpass_users[0]["name"],
    "role": "user",
    "channel": "chat",
    "tags": "login, access",
}

In [33]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()

    if not user:
        user = udahub.User(
            user_id=str(uuid.uuid4()),
            account_id=account_id,
            external_user_id=ticket_info["owner_id"],
            user_name=ticket_info["owner_name"],
        )
    
    ticket = udahub.Ticket(
        ticket_id=str(uuid.uuid4()),
        account_id=account_id,
        user_id=user.user_id,
        channel=ticket_info["channel"],
    )
    metadata = udahub.TicketMetadata(
        ticket_id=ticket.ticket_id,
        status=ticket_info["status"],
        main_issue_type=None,
        tags=ticket_info["tags"],
    )

    first_message = udahub.TicketMessage(
        message_id=str(uuid.uuid4()),
        ticket_id=ticket.ticket_id,
        role=ticket_info["role"],
        content=ticket_info["content"],
    )

    session.add_all([user, ticket, metadata, first_message])


# Tests

In [34]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    print(account)

<Account(account_id='cultpass', account_name='CultPass Card')>


In [35]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    for article in account.knowledge_articles:
        print(article)

<Knowledge(article_id='c9453428-6c98-4d98-9510-4c276cb31fba', title='How to Reserve a Spot for an Event')>
<Knowledge(article_id='a8291bea-fae8-43e3-9905-3089850c6e89', title='What's Included in a CultPass Subscription')>
<Knowledge(article_id='1f7f2799-2912-4ca9-98cc-43d5d3159b84', title='How to Cancel or Pause a Subscription')>
<Knowledge(article_id='29c400d3-7841-4184-bc70-16e46eaed7d7', title='How to Handle Login Issues?')>
<Knowledge(article_id='d0f2006e-dce4-4a86-b7bd-9604e2268a8e', title='Understanding Quotas and Rollovers')>
<Knowledge(article_id='7de5f3ed-c580-4b71-bea6-f945242d2032', title='Update App and Clear Cache')>
<Knowledge(article_id='5a4e6158-02dc-4f52-bcc3-28778206b28f', title='Change Password and Manage 2FA')>
<Knowledge(article_id='080601e9-5833-4d70-a0b3-27384e4987af', title='Payment Methods and Receipts')>
<Knowledge(article_id='f6296a98-4c99-4fbe-883d-5e2b9931cbde', title='Refund Eligibility')>
<Knowledge(article_id='bc4abf48-ea82-4e50-a1d2-78cb2eaf8032', title

In [36]:
with get_session(engine) as session:
    users = session.query(udahub.User).all()
    for user in users:
        print(user)

<User(user_id='88e5da8d-1909-4571-aada-cc97bf247ecb', user_name='Alice Kingsley', external_user_id='a4ab87')>


In [37]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()
    
    ticket:udahub.Ticket = user.tickets[0]
    for message in ticket.messages:
        print(message)

<TicketMessage(message_id='4af5d361-0d03-45b8-8926-de6e8eeb7246', role='user', content='I can't log in to my Cultpass ...')>


In [39]:
import os
import sqlite3

# Check if the database exists
db_path = "udahub.db"
if os.path.exists(db_path):
    print("Database exists")
    
    # Check articles
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Check if knowledge_articles table exists
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='knowledge_articles'")
    if cursor.fetchone():
        cursor.execute("SELECT COUNT(*) FROM knowledge_articles")
        count = cursor.fetchone()[0]
        print(f"Number of articles: {count}")
        
        if count > 0:
            cursor.execute("SELECT title FROM knowledge_articles LIMIT 3")
            titles = cursor.fetchall()
            print("Sample titles:", [t[0] for t in titles])
    else:
        print("knowledge_articles table does not exist")
    
    conn.close()
else:
    print("Database does not exist - need to run setup notebook")

Database does not exist - need to run setup notebook
